In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score
from support.neural_nets import *
from sklearn.preprocessing import OneHotEncoder
# import MNIST from tensorflow

mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28**2)
x_test = x_test.reshape(-1, 28**2)
# from ag's workbook
x_train = x_train.astype(np.float32).reshape(-1, 28*28) / 255.0
x_test = x_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

# y_train = y_train.reshape(-1, 1)
# y_test = y_test.reshape(-1, 1)

# oh = OneHotEncoder(sparse=False)
# oh.fit(y_train)
# y_train = oh.transform(y_train)
# y_test = oh.transform(y_test)

C:\Users\tdele\Anaconda3\envs\ml\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((60000, 784), (60000,), (10000, 784), (10000,))

In [4]:
n_inputs = x_train.shape[1] # replace by getting actual inputs from X data
n_classes = 10 # same as above

In [5]:
from datetime import datetime
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

learning_rate=0.01
tf.reset_default_graph()

# Construction phase
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')
y = tf.placeholder(tf.int32, shape=(None), name='y')

h1_units = 400
h2_units = 300
h3_units = 225

with tf.name_scope('dnn'):
    hidden1 = tf.layers.dense(X, h1_units, activation=tf.nn.relu, name='hidden1')
    hidden2 = tf.layers.dense(hidden1, h2_units, activation=tf.nn.relu, name='hidden2')
    hidden3 = tf.layers.dense(hidden2, h3_units, activation=tf.nn.relu, name='hidden3')
    
    logits = tf.layers.dense(hidden3, n_classes, name='outputs')

with tf.name_scope('loss'):
    xn = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits, name='cross_entropy')
    loss = tf.reduce_mean(xn)
    loss_summary = tf.summary.scalar('Loss', loss)
    
with tf.name_scope('training'):
    opt = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train_op = opt.minimize(loss)
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1, name='correct')
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')
    acc_summary = tf.summary.scalar('Accuracy', accuracy)

init = tf.global_variables_initializer()
saver = tf.train.Saver()
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

![Image of the Graph](images/tf_graph_img.png)

In [9]:
# Execution phase
epochs = 150
batch_size = 500
n_batches = int(np.ceil(x_train.shape[0] / batch_size))

with tf.Session() as sess:
    init.run()
    
    for epoch in range(epochs):
        if epoch % 10 == 0:
            saver.save(sess, "./tmp/dnn/dnn_ckpt_{}.ckpt".format(epoch))
        for batch, (X_batch, y_batch) in enumerate(fetch_batch(x_train, y_train, batch_size)):
            sess.run(train_op, feed_dict={X: X_batch, y: y_batch})
            if batch % 10 == 0:
                acc_train, loss_train = sess.run([accuracy, loss],
                                                 feed_dict={X: x_train, y: y_train})
                acc_test, loss_test, acc_summary_str, loss_summary_str = sess.run([accuracy, loss, acc_summary, loss_summary],
                                                                                  feed_dict={X: x_test, y: y_test})
                step = epoch * n_batches + batch
                file_writer.add_summary(acc_summary_str, step)
                file_writer.add_summary(loss_summary_str, step)
                
        print("Epoch {}:\tTrain accuracy: {:.2%}\tTest accuracy: {:.2%}".format(epoch, acc_train, acc_test))
    save_path = saver.save(sess, "./dnn_final.ckpt")

Epoch 0:	Train accuracy: 68.50%	Test accuracy: 69.53%
Epoch 1:	Train accuracy: 81.49%	Test accuracy: 82.04%
Epoch 2:	Train accuracy: 84.82%	Test accuracy: 85.22%
Epoch 3:	Train accuracy: 87.05%	Test accuracy: 87.54%
Epoch 4:	Train accuracy: 88.28%	Test accuracy: 88.68%
Epoch 5:	Train accuracy: 89.07%	Test accuracy: 89.39%
Epoch 6:	Train accuracy: 89.74%	Test accuracy: 90.13%
Epoch 7:	Train accuracy: 90.17%	Test accuracy: 90.46%
Epoch 8:	Train accuracy: 90.55%	Test accuracy: 90.75%
Epoch 9:	Train accuracy: 90.88%	Test accuracy: 91.11%
Epoch 10:	Train accuracy: 91.25%	Test accuracy: 91.40%
Epoch 11:	Train accuracy: 91.49%	Test accuracy: 91.61%
Epoch 12:	Train accuracy: 91.77%	Test accuracy: 91.88%
Epoch 13:	Train accuracy: 91.96%	Test accuracy: 92.14%
Epoch 14:	Train accuracy: 92.10%	Test accuracy: 92.27%
Epoch 15:	Train accuracy: 92.36%	Test accuracy: 92.53%
Epoch 16:	Train accuracy: 92.54%	Test accuracy: 92.72%
Epoch 17:	Train accuracy: 92.66%	Test accuracy: 92.83%
Epoch 18:	Train accu

Epoch 149:	Train accuracy: 98.57%	Test accuracy: 97.38%


![Accuracy](images/tf_accuracy_plot.png)